In [1]:
from interface import ObjectInterface
from ddl import Interpreter
import utils
import logging
import os
import examples.randomdata as rd
from decimal import Decimal
import random
from datetime import datetime

FILENAME_DATABASE = 'data/database.db'
FILENAME_STRUCTURE = 'setup/example_structure.ddl'
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.DEBUG)

def get_interface() -> ObjectInterface:
    interface = ObjectInterface(FILENAME_DATABASE)
    interface.connect()
    return interface

### Datenbank neu aufsetzen und Datenmodell einspielen

In [2]:
# Datenbank löschen (wenn existiert)
if os.path.exists(FILENAME_DATABASE):
    os.remove(FILENAME_DATABASE)

# Datenmodell einspielen
with get_interface() as interface:
    interface.setup()
    interpreter = Interpreter(interface)
    with open(FILENAME_STRUCTURE, 'r') as file:
        interpreter.run(file.read())
    interface.commit()

[2024-02-13 07:16:19] DEBUG: Setup successful
[2024-02-13 07:16:19] DEBUG: Created datatype void (NULL, no read transformer, no write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype int (INTEGER, no read transformer, no write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype shorttext (VARCHAR(64), no read transformer, no write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype longtext (VARCHAR(256), no read transformer, no write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype text (TEXT, no read transformer, no write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype date (DATE, read transformer, write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype datetime (DATETIME, read transformer, write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype zipcode (VARCHAR(5), no read transformer, no write transformer)
[2024-02-13 07:16:19] DEBUG: Created datatype currency2 (INTEGER, read transformer, write transformer)
[2024-02-13 

### Kunden einfügen

In [3]:
rd.set_seed(2024)

def get_random_customer():
    customer = rd.get_random_person()
    customer.update(rd.get_random_address())
    return customer

print(get_random_customer())

{'first_name': 'Ottilie', 'last_name': 'Koch', 'birthday': datetime.date(2008, 9, 18), 'street': 'Schröderstraße', 'house_number': '455c', 'zip': '44763', 'city': 'Fuchsdorf'}


In [4]:
n_customers = 100
customer_ids = []
with get_interface() as interface:
    for i in range(n_customers):
        customer_ids.append(interface.create_object('Customer', **get_random_customer()).id)
    interface.commit()
print(f"Inserted {n_customers} customers")

Inserted 100 customers


### Produkte einfügen

In [5]:
product_ids = []
products = [
    ['Rennrad', Decimal('995.99')],
    ['BMX', Decimal('195.99')],
    ['Hollandrad', Decimal('249.99')],
    ['Tourenrad', Decimal('549.99')],
    ['Kette', Decimal('19.99')],
    ['Klingel', Decimal('5.99')],
    ['Schlauch 28 Zoll', Decimal('7.99')],
    ['Schlauch 26 Zoll', Decimal('7.49')],
    ['Reifen 28 Zoll', Decimal('18.99')],
    ['Reifen 26 Zoll', Decimal('17.99')],
    ['Kettenfett', Decimal('3.99')],
    ['Helm', Decimal('144.99')]
]
with get_interface() as interface:
    for p in products:
        product_ids.append(interface.create_object('Product', name=p[0], price=p[1]).id)
    interface.commit()
print(f"Inserted {len(products)} products")

Inserted 12 products


### Bestellungen einfügen

In [6]:
n_orders = 40
order_ids = []
with get_interface() as interface:
    for i in range(n_orders):
        n_products = random.randint(1, 10)

        # Bestellung erstellen und Kunde verknüpfen
        customer = interface.get_object(random.choice(customer_ids))
        order = interface.create_object('Order', creation_time=datetime.now())
        order.bind('order_to_customer', [customer])

        # Positionen verknüpfen
        positions = []
        for k in range(n_products):
            product = interface.get_object(random.choice(product_ids))
            position = interface.create_object('OrderPosition', amount=random.randint(1, 10))
            position.bind('position_to_product', [product])
            positions.append(position)
        order.bind('order_to_positions', positions)
        order_ids.append(order.id)

        print(f"{customer['full_name']} bought {n_products} products")
    interface.commit()

Ottilie Fischer bought 10 products
Heidi Hahn bought 6 products
Doris Schmidt bought 6 products
Melanie Koch bought 8 products
Uwe Schmid bought 1 products
Quentin Richter bought 9 products
Vivien Bauer bought 7 products
Clemens Becker bought 8 products
Gerhard Schröder bought 8 products
Franz Lehmann bought 10 products
Stefan Wolf bought 7 products
Quentin Schäfer bought 4 products
Gerhard Hahn bought 7 products
Anne Peters bought 6 products
Nicolas König bought 8 products
Vivien Lang bought 1 products
Emil Fuchs bought 3 products
Bert Schmid bought 3 products
Julian Neumann bought 1 products
Walter Lang bought 2 products
Inga Braun bought 2 products
Katrin Huber bought 10 products
Clemens Krüger bought 9 products
Ottilie Schmitt bought 2 products
Uwe Scholz bought 3 products
Inga Braun bought 2 products
Tanja Peters bought 2 products
Ottilie Schmitt bought 10 products
Katrin Huber bought 9 products
Ottilie Schmitt bought 8 products
Clemens Becker bought 9 products
Franz Fuchs bought 

### Bestellung anzeigen

In [7]:
with get_interface() as interface:
    # Bestellung
    order = interface.get_object(random.choice(order_ids))
    print(order.dump(), '\n')

    # Kunde
    customer = order.hop('order_to_customer')[0]
    print(customer.dump(), '\n')

    # Produkte
    positions = order.hop('order_to_positions')
    for position in positions:
        print(f"{position['amount']}x {position.hop_first('position_to_product')['name']}")

    df = positions.hop('position_to_product').get_dataframe()
    df_pos = positions.get_dataframe()
    expensive_positions = positions.filter(positions.get_column('price') > 150)
    df_expensive = expensive_positions.get_dataframe()

Order 124:
  price = 2930.69
  creation_time = 2024-02-13 07:16:19 

Customer 76:
  first_name = Heidi
  last_name = Hahn
  full_name = Heidi Hahn
  birthday = 1982-11-27
  street = Lehmannstraße
  house_number = 264
  zip = 15036
  city = Werneringen 

8x BMX
4x Schlauch 28 Zoll
1x Hollandrad
7x Helm
2x Klingel
9x Klingel


In [8]:
df_expensive.head(20)

,price,amount
id,,
125,1567.92,8
127,249.99,1
128,1014.93,7
